In [2]:
from scraping.yfinance_scraper import fetch_yfinance_news_data

In [3]:
from datetime import datetime, timedelta

In [4]:
end_datetime = datetime.now() - timedelta(days=30)
end_datetime

datetime.datetime(2025, 5, 22, 21, 43, 57, 247252)

In [6]:
ticker = "NVDA"

In [7]:
nvda_news = fetch_yfinance_news_data(ticker, end_datetime, 100, slp_time=10)

Scrolling 9...Total Srcoll: 9
News Collecting...
News collected from 2025-06-21 06:02:00 to 2025-06-16 17:15:00


In [10]:
nvda_news.tail()

,Datetime,headline,description,article,link
193,2025-06-16 17:30:00,Is This Beaten-Down Cathie Wood Artificial Int...,Recursion could help revolutionize the way dru...,Recursion could help revolutionize the way dru...,https://finance.yahoo.com/news/beaten-down-cat...
194,2025-06-16 17:30:00,Big Tech’s Furious Rally Forces Options Pros t...,(Bloomberg) -- Big Tech has led the furious re...,(Bloomberg) -- Big Tech has led the furious re...,https://finance.yahoo.com/news/big-tech-furiou...
195,2025-06-16 17:30:00,AI Companies Should Be Wary of Gulf Spending S...,Nvidia and other U.S. tech companies are benef...,Nvidia and other U.S. tech companies are benef...,https://finance.yahoo.com/m/0be225d1-5cd7-3fce...
196,2025-06-16 17:25:00,"Nvidia, Deutsche Telekom to launch industrial ...","Deutsche Telekom will oversee the AI factory, ...",US-based tech giant Nvidia has collaborated wi...,https://finance.yahoo.com/news/nvidia-deutsche...
197,2025-06-16 17:15:00,"The Best Stocks to Buy With $1,000 Right Now",Nvidia's growth is far from complete. Taiwan ...,Nvidia's growth is far from complete..Taiwan S...,https://finance.yahoo.com/news/best-stocks-buy...


In [11]:
nvda_news.to_csv(f"./data/yf_news_{ticker}.csv", index=False)

In [12]:
descriptions = nvda_news['description']

In [2]:
import os
# from langchain_community.document_loaders import TextLoader
from langchain_community.llms import Ollama
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableMap
import pandas as pd

In [4]:
nvda_news = pd.read_csv(f"./data/yf_news_NVDA.csv")
nvda_news.head()

,Datetime,headline,description,article,link
0,2025-06-21 06:02:00,"Stock market today: S&P 500, Nasdaq fall amid ...",Traders are weighing the possibility of direct...,US stocks closed the session mixed on Friday a...,https://finance.yahoo.com/news/live/stock-mark...
1,2025-06-21 19:51:00,"If I Could Buy Only 1 ""Magnificent 7"" Stock Ov...",Alphabet stock is lagging behind other Magnifi...,Alphabet shares have dipped 2% over the past y...,https://finance.yahoo.com/news/could-buy-only-...
2,2025-06-21 19:00:00,CrowdStrike Just Partnered Up With Nvidia. Sho...,A powerful alliance with Nvidia is redefining ...,CrowdStrike (CRWD) continues to ride the wave ...,https://finance.yahoo.com/news/crowdstrike-jus...
3,2025-06-21 19:00:00,"Nvidia Stock Prediction, From Someone Who Has ...",The big winner of the first phase of the artif...,The big winner of the first phase of the artif...,https://finance.yahoo.com/news/nvidia-stock-pr...
4,2025-06-21 19:20:00,10 Monster Stocks to Hold for the Next 10 Years,"Companies like Nvidia, Broadcom, and TSMC are ...","Companies like Nvidia, Broadcom, and TSMC are ...",https://finance.yahoo.com/news/10-monster-stoc...


In [5]:
!ollama run deepseek-r1

^C


In [6]:
llm = ChatOllama(model="deepseek-r1")

In [7]:


prompt = PromptTemplate.from_template(
    """
    You are a professional financial agent for risk management. 
    You will be given the headline and the description of a news.
    Analysis the risk of the news of the article reflecting.ollama run deepseek-r1
    \n\n{headline}
    \n\n{description}
    Just tell the type of risk you have classified. If there is no risk, say no risk.
    """
)



In [8]:


summarization_chain = (
    prompt
    | llm
    | StrOutputParser()
)



In [9]:
headline = nvda_news['headline'][0]
description = nvda_news['description'][0]
headline, description

('Stock market today: S&P 500, Nasdaq fall amid Fed rate uncertainty as Trump mulls Iran move',
 'Traders are weighing the possibility of direct US military involvement in the Israel-Iran war, now in its second week.')

In [10]:
result = summarization_chain.invoke({'headline': headline, 'description':description})

In [12]:
result.split('/think>\n')[-1]

'geopolitical'